<h1 align="center"><font size="5">Segmenting and Clustering Neighborhoods in Toronto</font></h1>  

<a id='ToC'></a>  

<div class="alert alert-block alert-info" style="margin-top: 20px">
    
# Table of Contents
1. [Scrape Canadian Postal Code](#FirstSection)  
     1.1. [BeautifulSoup](#11)  
     1.2. [Preprocessing](#12)  
     1.3. [Clean DataFrame](#13)  
2. [Obtain the Latitude and Longitude coordinates](#SecondSection)  
     2.1. [Using package - did not work](#21)  
     2.2. [Using csv](#22)  
     2.3. [Latitude Longitude DataFrame - Final](#23)  
3. [Explore and cluster the Toronto Neighborhood](#ThirdSection)  
     3.1. [Following the NY Cluster lab](#31)  
     3.2. [Subsetting to East, Central, Downtown, and West Toronto](#32)  
     3.3. [Decoupling Neighborhood](#33)  
     3.4. [Map of East, Central, Downtown, and West Toronto Neighborhood](#34)  
     3.5. [FourSquare API](#35)  
     3.6. [Preprocessing Neighborhood](#36)  
     3.7. [K-Means Clustering](#37)  
     3.8. [Cluster Map](#38)  
     3.9. [Examine Cluster](#39)  
</div>

<div class="alert alert-block alert-success" style="margin-top: 20px">

<a id='FirstSection'> <H1> First Assignment - Scrape Canadian Postal Code</H1></a>  
     1.1. [BeautifulSoup](#11)  
     1.2. [Preprocessing](#12)  
     1.3. [Clean DataFrame](#13)  
</div>

Let's import a couple dependancies before we start:

In [1]:
# Segmenting and Clustering Neighborhoods in Toronto
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

<a id='11'> BeautifulSoup</H1></a>

First things first: let's scrape the wikipedia page, using BeautifulSoup package

In [2]:
from bs4 import BeautifulSoup

In [3]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')

In [4]:
table = soup.find('table',class_='wikitable sortable')

In [5]:
table.prettify().splitlines()

['<table class="wikitable sortable">',
 ' <tbody>',
 '  <tr>',
 '   <th>',
 '    Postcode',
 '   </th>',
 '   <th>',
 '    Borough',
 '   </th>',
 '   <th>',
 '    Neighbourhood',
 '   </th>',
 '  </tr>',
 '  <tr>',
 '   <td>',
 '    M1A',
 '   </td>',
 '   <td>',
 '    Not assigned',
 '   </td>',
 '   <td>',
 '    Not assigned',
 '   </td>',
 '  </tr>',
 '  <tr>',
 '   <td>',
 '    M2A',
 '   </td>',
 '   <td>',
 '    Not assigned',
 '   </td>',
 '   <td>',
 '    Not assigned',
 '   </td>',
 '  </tr>',
 '  <tr>',
 '   <td>',
 '    M3A',
 '   </td>',
 '   <td>',
 '    <a href="/wiki/North_York" title="North York">',
 '     North York',
 '    </a>',
 '   </td>',
 '   <td>',
 '    <a href="/wiki/Parkwoods" title="Parkwoods">',
 '     Parkwoods',
 '    </a>',
 '   </td>',
 '  </tr>',
 '  <tr>',
 '   <td>',
 '    M4A',
 '   </td>',
 '   <td>',
 '    <a href="/wiki/North_York" title="North York">',
 '     North York',
 '    </a>',
 '   </td>',
 '   <td>',
 '    <a href="/wiki/Victoria_Villa

In [6]:
# Convert the html into pandas dataframe:
df = pd.read_html(str(table),header=0)[0]

<a id='12'> Pre-processing</a>  
* The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [7]:
# rename the column to the one specified by the assignment
df.columns = ['PostalCode','Borough','Neighborhood']

In [8]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [9]:
# sanity check:
df[df.PostalCode=='M2H']

,PostalCode,Borough,Neighborhood
63,M2H,North York,Hillcrest Village


* Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [10]:
# remove cells with a borough that is "Not assigned"
df = df[df.Borough!='Not assigned']

In [11]:
# to check whether all the 'not assigned' has been dropped:
sum(df.Borough=='Not assigned')

0

In [12]:
df.shape

(211, 3)

* More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [13]:
# split the data frame into: duplicated and unique df. 
duplicated_df = df[df.duplicated(subset='PostalCode',keep=False)].sort_values(by='PostalCode')
duplicated_df.head()

,PostalCode,Borough,Neighborhood
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern
28,M1C,Scarborough,Rouge Hill
29,M1C,Scarborough,Port Union
27,M1C,Scarborough,Highland Creek


In [14]:
duplicated_df.shape

(165, 3)

Let's also separate the dataframe with unique ```PostalCode```. Once the cleaning up of the duplicated ```PostalCode``` is done, we can then add them back in. 

In [15]:
unique_df = df[~df.duplicated(subset='PostalCode',keep=False)].sort_values(by='PostalCode')

In [16]:
unique_df.shape

(46, 3)

In [17]:
unique_df[unique_df.PostalCode=='M2H']

,PostalCode,Borough,Neighborhood
63,M2H,North York,Hillcrest Village


There are 211 rows for the ```df``` which had been separated into:  
* ```duplicated_df``` with 165 rows
* ```unique_df``` with 46 rows

Let's clean up the ```duplicated_df``` data frame:

In [18]:
# grab the first duplicated value using the ~ duplicated below:
cleaned_dup_df = duplicated_df[~duplicated_df.duplicated(subset='PostalCode')]
cleaned_dup_df.head()

,PostalCode,Borough,Neighborhood
11,M1B,Scarborough,Rouge
28,M1C,Scarborough,Rouge Hill
44,M1E,Scarborough,West Hill
93,M1K,Scarborough,Kennedy Park
107,M1L,Scarborough,Clairlea


In [19]:
cleaned_dup_df.shape

(57, 3)

In [20]:
# extracting the neighborhood list from the cleaned_dup_df and grabbing them from the 
# duplicated_df dataframe using groupby and get_group method
neighborhood_list = []
for postal in cleaned_dup_df['PostalCode']:
    neighborhood_list.append(list(duplicated_df.groupby('PostalCode').get_group(postal)['Neighborhood']))

In [21]:
neighborhood_list[:5]

[['Rouge', 'Malvern'],
 ['Rouge Hill', 'Port Union', 'Highland Creek'],
 ['West Hill', 'Morningside', 'Guildwood'],
 ['Kennedy Park', 'Ionview', 'East Birchmount Park'],
 ['Clairlea', 'Golden Mile', 'Oakridge']]

In [22]:
# to flatten the list, use list comprehension trick:
neighborhood_list = [', '.join(i) for i in neighborhood_list]
neighborhood_list[:5]

['Rouge, Malvern',
 'Rouge Hill, Port Union, Highland Creek',
 'West Hill, Morningside, Guildwood',
 'Kennedy Park, Ionview, East Birchmount Park',
 'Clairlea, Golden Mile, Oakridge']

In [23]:
# add a new column into the dataframe to hold the new list of multiple neighborhood in one row
cleaned_dup_df.insert(2,'Neighborhood_dup',neighborhood_list)

In [24]:
# remove the old neighborhood column:
cleaned_dup_df = cleaned_dup_df.drop('Neighborhood',axis=1)

In [25]:
# rename the 'Neighborhood_dup' with 'Neighborhood'
cleaned_dup_df.columns = ['PostalCode','Borough','Neighborhood']
cleaned_dup_df.head()

,PostalCode,Borough,Neighborhood
11,M1B,Scarborough,"Rouge, Malvern"
28,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
44,M1E,Scarborough,"West Hill, Morningside, Guildwood"
93,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
107,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"


In [26]:
# combine the unique_df and cleaned_dup_df with concat function:
cleaned_df = pd.concat([cleaned_dup_df,unique_df])

In [27]:
# sort the value and reset the old index to the new index
cleaned_df.sort_values(by='PostalCode')
cleaned_df = cleaned_df.reset_index(drop=True)

In [28]:
# For sanity check:
cleaned_df[cleaned_df.PostalCode=='M5A']

,PostalCode,Borough,Neighborhood
24,M5A,Downtown Toronto,"Harbourfront, Regent Park"


* If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [29]:
# figured out where the "Not assigned" row in the new cleaned_df located
cleaned_df[cleaned_df.Neighborhood=='Not assigned']

,PostalCode,Borough,Neighborhood
95,M7A,Queen's Park,Not assigned


In [30]:
# rename the neighborhood into "Queen's Park"
cleaned_df.loc[95,'Neighborhood'] = "Queen's Park"

In [31]:
# For sanity check: compare it with the list of postal code given from this assignment
cleaned_df[cleaned_df['PostalCode'].isin(['M5G','M2H','M4B','M1J','M4G','M4M','M1R','M9V','M9L','M5V','M1B','M7A'])]

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
8,M1R,Scarborough,"Maryvale, Wexford"
18,M4B,East York,"Parkview Hill, Woodbine Gardens"
35,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo..."
56,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."
59,M1J,Scarborough,Scarborough Village
63,M2H,North York,Hillcrest Village
76,M4G,East York,Leaside
79,M4M,East Toronto,Studio District
88,M5G,Downtown Toronto,Central Bay Street


<a id='13'> Clean DataFrame</a>  
* Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.

In [32]:
# Here is the finished dataframe:
cleaned_df

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"West Hill, Morningside, Guildwood"
3,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
4,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
5,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
6,M1N,Scarborough,"Birch Cliff, Cliffside West"
7,M1P,Scarborough,"Wexford Heights, Dorset Park, Scarborough Town..."
8,M1R,Scarborough,"Maryvale, Wexford"
9,M1T,Scarborough,"Clarks Corners, Sullivan, Tam O'Shanter"


* In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [33]:
cleaned_df.shape

(103, 3)

<div class="alert alert-block alert-success" style="margin-top: 20px">  
    
<a id='SecondSection'> <H1> Second Assignment - Obtain the Latitude and Longitude coordinates </H1></a>  
    2.1. [Using package - did not work](#21)  
    2.2. [Using csv](#22)  
    2.3. [Latitude Longitude DataFrame - Final](#23)  

[Back to the top](#ToC)  

</div>

In [34]:
## this is how I would have used the geocoder
## unfortunately, the code did not work 
#import geocoder # import geocoder
#lat_lng = []
#for postal_code in cleaned_df['PostalCode']:
#    lat_lng_coords = None
#    # loop until you get the coordinates
#    while(lat_lng_coords is None):
#        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#        lat_lng_coords = g.latlng
#    lat_lng.append(lat_lng_coords)

<a id='21'>Using package - did not work</a>  

In [34]:
# Instead, I am going to use the pypostalcode package:
# example using'M5G':
from pypostalcode import PostalCodeDatabase
pcdb = PostalCodeDatabase()
pc = 'M5G'
location = pcdb[pc]
print("latitude for M5G:",location.latitude)
print("longitude for M5G:",location.longitude)

latitude for M5G: 43.6519
longitude for M5G: -79.3874


In [35]:
# let's create a for loop and getting all the data into one:
pcdb = PostalCodeDatabase()
lat_ca = []
long_ca = []
for postal_code in cleaned_df['PostalCode']:
    try:
        location = pcdb[postal_code]
    except:
        location.latitude = np.nan
        location.longitude = np.nan
    lat_ca.append(location.latitude)
    long_ca.append(location.longitude)
# let's insert the latitude column into the cleaned_df:
cleaned_df.insert(3,'Latitude',lat_ca)
# let's insert the latitude column into the cleaned_df:
cleaned_df.insert(4,'Longitude',long_ca)
# Check one row that is missing:
cleaned_df[cleaned_df['Latitude'].isnull()]

,PostalCode,Borough,Neighborhood,Latitude,Longitude
96,M7R,Mississauga,Canada Post Gateway Processing Centre,NaN,NaN


In [36]:
# to get the value, we will use the csv file given by the IBM coursera course:
downloaded_df = pd.read_csv('Geospatial_Coordinates.csv')
downloaded_df.head()
# let's make sure that the column name is the same as what we had in cleaned_df:
downloaded_df.columns = ['PostalCode','Latitude','Longitude']
# from the downloaded_df:
# downloaded_df[downloaded_df['Postal Code']=='M7R']
# let's replace the NaN value for M7R:
cleaned_df.loc[cleaned_df['Latitude'].isnull(),'Latitude'] = float(downloaded_df[downloaded_df['PostalCode']=='M7R']['Latitude'])
cleaned_df.loc[cleaned_df['Longitude'].isnull(),'Longitude'] = float(downloaded_df[downloaded_df['PostalCode']=='M7R']['Longitude'])
cleaned_df.loc[cleaned_df.PostalCode=='M7R']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
96,M7R,Mississauga,Canada Post Gateway Processing Centre,43.636966,-79.615819


In [37]:
# For sanity check: compare it with the list of postal code given from this assignment
cleaned_df[cleaned_df['PostalCode'].isin(['M5G','M2H','M4B','M1J','M4G','M4M','M1R','M9V','M9L','M5V','M1B','M5A'])]

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.7976,-79.2270
8,M1R,Scarborough,"Maryvale, Wexford",43.7293,-79.3038
18,M4B,East York,"Parkview Hill, Woodbine Gardens",43.6979,-79.2986
24,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.6369,-79.3505
35,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.6525,-79.3686
56,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.7300,-79.5542
59,M1J,Scarborough,Scarborough Village,43.7315,-79.2460
63,M2H,North York,Hillcrest Village,43.7895,-79.3735
76,M4G,East York,Leaside,43.6918,-79.3708
79,M4M,East Toronto,Studio District,43.6505,-79.3369


<a id='22'> Using csv</a>  
The value doesn't exactly aligned with the data provided by the coursera assignment, so I will instead use the downloaded_df value:

In [38]:
# let's create a new dataframe and drop the Latitude and Longitude columns from the dataframe
cleaned_df2 = cleaned_df.copy()
cleaned_df2.drop(['Latitude','Longitude'],axis=1,inplace=True)
cleaned_df2.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"West Hill, Morningside, Guildwood"
3,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
4,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"


In [39]:
# merge cleaned_df2 with downloaded_df to add latitude/longitude for each PostalCode
cleaned_df2 = cleaned_df2.join(downloaded_df.set_index('PostalCode'), on='PostalCode')

<a id='23'>Latitude Longitude DataFrame - Final</a>

In [40]:
# For sanity check: compare it with the list of postal code given from this assignment
cleaned_df2[cleaned_df2['PostalCode'].isin(['M5G','M2H','M4B','M1J','M4G','M4M','M1R','M9V','M9L','M5V','M1B','M5A'])]

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
8,M1R,Scarborough,"Maryvale, Wexford",43.750072,-79.295849
18,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
24,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
35,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420
56,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
59,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
63,M2H,North York,Hillcrest Village,43.803762,-79.363452
76,M4G,East York,Leaside,43.709060,-79.363452
79,M4M,East Toronto,Studio District,43.659526,-79.340923


This value is the same as the one given from the coursera assignment. 

In [41]:
# To show the whole value:
cleaned_df2

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"West Hill, Morningside, Guildwood",43.763573,-79.188711
3,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
4,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
5,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
6,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
7,M1P,Scarborough,"Wexford Heights, Dorset Park, Scarborough Town...",43.757410,-79.273304
8,M1R,Scarborough,"Maryvale, Wexford",43.750072,-79.295849
9,M1T,Scarborough,"Clarks Corners, Sullivan, Tam O'Shanter",43.781638,-79.304302


<div class="alert alert-block alert-success" style="margin-top: 20px">  
    
<a id='ThirdSection'> <H1> Third Assignment - Explore and cluster the Toronto Neighborhood</H1></a>  
     3.1. [Following the NY Cluster lab](#31)  
     3.2. [Subsetting to East, Central, Downtown, and West Toronto](#32)  
     3.3. [Decoupling Neighborhood](#33)  
     3.4. [Map of East, Central, Downtown, and West Toronto Neighborhood](#34)  
     3.5. [FourSquare API](#35)  
     3.6. [Preprocessing Neighborhood](#36)  
     3.7. [K-Means Clustering](#37)  
     3.8. [Cluster Map](#38)  
     3.9. [Examine Cluster](#39)  

[Back to the top](#ToC)  
</div>

<a id='31'>Following the NY cluster lab</a>  
Just checking on the number of boroughs and neighborhoods from cleaned_df2 and df dataframe:

In [42]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(cleaned_df2['Borough'].unique()),
        len(cleaned_df2['Neighborhood'].unique())
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>toronto_explorer</em>, as shown below.

In [43]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [44]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(cleaned_df2['Latitude'], cleaned_df2['Longitude'], cleaned_df2['Borough'], cleaned_df2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

There are multiple neighborhood per marker. It will be decoupled in the next couple of lines.

Let's explore the Boroughs in the dataframe:

In [45]:
cleaned_df2['Borough'].unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'West Toronto', 'York',
       'Etobicoke', "Queen's Park", 'Mississauga'], dtype=object)

<a id='32'>Subsetting to East, Central, Downtown, and West Toronto</a>  
[Back to Section 3](#ThirdSection)    
We are going to subset the data and only specify the 'East Toronto', 'Central Toronto', 'Downtown Toronto' , and 'West Toronto' Borough using ```str.contains```:

In [46]:
toronto_data = cleaned_df2[cleaned_df2['Borough'].str.contains('Toronto')]
# reset the index:
toronto_data = toronto_data.reset_index(drop=True)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
1,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
2,M4T,Central Toronto,"Summerhill East, Moore Park",43.689574,-79.383160
3,M4V,Central Toronto,"Rathnelly, Deer Park, South Hill, Summerhill W...",43.686412,-79.400049
4,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675


In [47]:
# shape of the data:
toronto_data.shape

(38, 5)

<a id='33'>Decoupling Neighborhood</a>  
[Back to Section 3](#ThirdSection)    
There are multiple neighborhood for each postal code, so I will decouple them and get the latitude and longitude using the Nominatim (geocode) function.

First let's check which rows have one neighborhood and which rows have multiple neighborhood:

In [48]:
len(toronto_data['Neighborhood'][0].split(','))

2

In [49]:
toronto_data.shape[0]

38

In [50]:
one_neigh= []
mult_neigh = []
for i in range(toronto_data.shape[0]):
    countN = len(toronto_data['Neighborhood'][i].split(','))
    if countN > 1:
        mult_neigh.append(i)
    elif countN == 1:
        one_neigh.append(i)

In [51]:
toronto_data_mult = toronto_data.loc[mult_neigh]
toronto_data_mult.shape

(23, 5)

In [52]:
toronto_data_one = toronto_data.loc[one_neigh]
toronto_data_one.shape

(15, 5)

In [53]:
toronto_data_one

,PostalCode,Borough,Neighborhood,Latitude,Longitude
23,M4E,East Toronto,The Beaches,43.676357,-79.293031
24,M4M,East Toronto,Studio District,43.659526,-79.340923
25,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
26,M4P,Central Toronto,Davisville North,43.712751,-79.390197
27,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
28,M4S,Central Toronto,Davisville,43.704324,-79.388790
29,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
30,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
31,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
32,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


We will keep the ```toronto_data_one``` DataFrame and decouple the ```toronto_data_mult``` DataFrame:

In [54]:
neighbor_list = []
for neigh in toronto_data_mult['Neighborhood']:
    neighbor_list.append(neigh)
neighbor_list

['The Danforth West, Riverdale',
 'India Bazaar, The Beaches West',
 'Summerhill East, Moore Park',
 'Rathnelly, Deer Park, South Hill, Summerhill West, Forest Hill SE',
 'Cabbagetown, St. James Town',
 'Harbourfront, Regent Park',
 'Ryerson, Garden District',
 'King, Adelaide, Richmond',
 'Harbourfront East, Toronto Islands, Union Station',
 'Toronto Dominion Centre, Design Exchange',
 'Victoria Hotel, Commerce Court',
 'Forest Hill West, Forest Hill North',
 'The Annex, Yorkville, North Midtown',
 'Harbord, University of Toronto',
 'Chinatown, Kensington Market, Grange Park',
 'CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara',
 'Underground city, First Canadian Place',
 'Dufferin, Dovercourt Village',
 'Trinity, Little Portugal',
 'Parkdale Village, Exhibition Place, Brockton',
 'High Park, The Junction South',
 'Parkdale, Roncesvalles',
 'Swansea, Runnymede']

In [55]:
new = [i.split(', ') for i in neighbor_list]
# https://stackoverflow.com/questions/952914/how-to-make-a-flat-list-out-of-list-of-lists
neighbor_list = [item for sublist in new for item in sublist]
neighbor_list[:5]

['The Danforth West',
 'Riverdale',
 'India Bazaar',
 'The Beaches West',
 'Summerhill East']

Let's use the `geopy` package using Nominatim geolocation service:

In [56]:
lat_to = []
long_to = []
for neigh in neighbor_list:
    address = "{}, Toronto, ON".format(neigh)
    geolocator = Nominatim(user_agent="toronto_explorer")
    location = geolocator.geocode(address)
    if location is not None:
        lat_to.append(location.latitude)
        long_to.append(location.longitude)
    else:
        lat_to.append(np.nan)
        long_to.append(np.nan)

Create a new dataframe entitled `neighborhoods`:

In [57]:
column_names = ['Neighborhood', 'Latitude', 'Longitude'] 
# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [58]:
neighborhoods['Neighborhood'] = neighbor_list
neighborhoods['Latitude']= lat_to
neighborhoods['Longitude']= long_to

In [59]:
neighborhoods.head()

,Neighborhood,Latitude,Longitude
0,The Danforth West,43.686360,-79.300316
1,Riverdale,43.665470,-79.352594
2,India Bazaar,43.672223,-79.323503
3,The Beaches West,43.671024,-79.296712
4,Summerhill East,43.681678,-79.390504


In [60]:
# let's see the null values:
neighborhoods[neighborhoods['Latitude'].isnull()]

,Neighborhood,Latitude,Longitude
42,Railway Lands,NaN,NaN


Cross-checking the `Railway Lands` neighborhoods with the toronto_data dataframe to see their values:

In [61]:
toronto_data.loc[toronto_data['Neighborhood'].str.contains('Railway Lands')]

,PostalCode,Borough,Neighborhood,Latitude,Longitude
15,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.39442


The *Railway Lands* are amongst other downtown Toronto neighborhood Postal Code. I am comfortable with deleting this one.

In [62]:
neighborhoods = neighborhoods.drop(neighborhoods[neighborhoods['Neighborhood'] == 'Railway Lands'].index)

In [63]:
# let's see the null values:
neighborhoods[neighborhoods['Latitude'].isnull()]

,Neighborhood,Latitude,Longitude


We remove all the ```null``` values

Next, let's re-add the Borough column by taking the `toronto_data_mult` DataFrame and creating a list comprehension:

In [64]:
borough_list = [list(toronto_data_mult.loc[toronto_data_mult['Neighborhood'].str.contains(neigh)]['Borough']) 
 for neigh in neighborhoods['Neighborhood']]

In [65]:
borough_list[:15]

[['East Toronto'],
 ['East Toronto'],
 ['East Toronto'],
 ['East Toronto'],
 ['Central Toronto'],
 ['Central Toronto'],
 ['Central Toronto'],
 ['Central Toronto'],
 ['Central Toronto'],
 ['Central Toronto'],
 ['Central Toronto'],
 ['Downtown Toronto'],
 ['Downtown Toronto'],
 ['Downtown Toronto', 'Downtown Toronto', 'Downtown Toronto'],
 ['Downtown Toronto']]

There are rows that have multiple values, so I will only take the first one using another list comprehension:

In [66]:
borough_list = [borough_list[i][0] for i in range(len(borough_list))]
# check the length of the borough list
len(borough_list)

58

In [68]:
borough_list[:15]

['East Toronto',
 'East Toronto',
 'East Toronto',
 'East Toronto',
 'Central Toronto',
 'Central Toronto',
 'Central Toronto',
 'Central Toronto',
 'Central Toronto',
 'Central Toronto',
 'Central Toronto',
 'Downtown Toronto',
 'Downtown Toronto',
 'Downtown Toronto',
 'Downtown Toronto']

In [67]:
# make sure that the dataframe has the same length
neighborhoods.shape

(58, 3)

In [69]:
# let's insert the borough value into neighborhoods dataframe
neighborhoods.insert(0,'Borough',borough_list)
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,East Toronto,The Danforth West,43.686360,-79.300316
1,East Toronto,Riverdale,43.665470,-79.352594
2,East Toronto,India Bazaar,43.672223,-79.323503
3,East Toronto,The Beaches West,43.671024,-79.296712
4,Central Toronto,Summerhill East,43.681678,-79.390504


Now let's re-add the `toronto_data_one` DataFrame into the `neighborhoods` DataFrame. In order to do so, I will drop the Postal Code column to match the columns with one another: 

In [70]:
toronto_data_one = toronto_data_one.drop('PostalCode',axis=1)

Now let's rejoin the toronto_data_one and the neighborhoods:

In [71]:
neighborhoods = pd.concat([neighborhoods,toronto_data_one])
neighborhoods.sort_values(by='Borough')
neighborhoods = neighborhoods.reset_index(drop=True)

In [72]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude
0,East Toronto,The Danforth West,43.686360,-79.300316
1,East Toronto,Riverdale,43.665470,-79.352594
2,East Toronto,India Bazaar,43.672223,-79.323503
3,East Toronto,The Beaches West,43.671024,-79.296712
4,Central Toronto,Summerhill East,43.681678,-79.390504
5,Central Toronto,Moore Park,43.690388,-79.383297
6,Central Toronto,Rathnelly,43.677472,-79.400460
7,Central Toronto,Deer Park,43.688090,-79.394093
8,Central Toronto,South Hill,43.677926,-79.405767
9,Central Toronto,Summerhill West,43.681678,-79.390504


As we did with all of Toronto, let's visualize the smaller subset and the neighborhoods in it:

In [73]:
# create map of Toronto using latitude and longitude values
map_subset = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'],neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_subset)  
    
map_subset

We see that there are two anomalies:
1. Underground city
2. Richmond

They are both from downtown Toronto Borough, but located outside the city. So, I will need to fix this right now:

In [74]:
# Underground City taken from toronto_data:
toronto_data.loc[toronto_data['Neighborhood'].str.contains('Underground city')]

,PostalCode,Borough,Neighborhood,Latitude,Longitude
16,M5X,Downtown Toronto,"Underground city, First Canadian Place",43.648429,-79.38228


In [75]:
# from the neighborhoods data frame:
neighborhoods.loc[neighborhoods['Neighborhood'].str.contains('Underground city')]

,Borough,Neighborhood,Latitude,Longitude
43,Downtown Toronto,Underground city,43.770145,-79.374863


We can try the geocoder and see where the error lies:

In [76]:
address = 'Underground city, Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
#latitude = location.latitude
#longitude = location.longitude
print('The geograpical coordinate are {}, {}.'.format(latitude, longitude))
print(location)

The geograpical coordinate are 43.653963, -79.387207.
The Underground, 794a, Sheppard Avenue East, Bayview Village, Don Valley North, North York, Toronto, Ontario, M2K 1C3, Canada


The query pointed towards North York Borough, instead of downtown Toronto. So, using google, I found the coordinate to be: 43.6516645, -79.3816759. 

In [77]:
underground_loc = neighborhoods.loc[neighborhoods['Neighborhood'].str.contains('Underground city')].index

In [78]:
# let's replace them with the value that I obtained from google maps:
neighborhoods.loc[underground_loc,'Latitude'] = 43.6516645
neighborhoods.loc[underground_loc,'Longitude'] = -79.3816759

In [79]:
neighborhoods.loc[underground_loc]

,Borough,Neighborhood,Latitude,Longitude
43,Downtown Toronto,Underground city,43.651665,-79.381676


Let's fix Richmond:

In [80]:
# Richmond taken from toronto_data:
toronto_data.loc[toronto_data['Neighborhood'].str.contains('Richmond')]

,PostalCode,Borough,Neighborhood,Latitude,Longitude
7,M5H,Downtown Toronto,"King, Adelaide, Richmond",43.650571,-79.384568


In [81]:
# from the neighborhoods data frame:
neighborhoods.loc[neighborhoods['Neighborhood'].str.contains('Richmond')]

,Borough,Neighborhood,Latitude,Longitude
19,Downtown Toronto,Richmond,43.812589,-79.26337


We can try the geocoder and see where the error lies:

In [82]:
address = 'Richmond, Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
print(location)

Richmond Park, Agincourt North, Scarborough North, Scarborough, Toronto, Golden Horseshoe, Ontario, Canada


It points toward Richmond Park in the borough of Scarborough, instead of the Richmond Street in downtown Toronto. So, let's modify the query into *'Richmond Street'*:

In [83]:
address = 'Richmond St, Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(location)
print('The geograpical coordinate are {}, {}.'.format(latitude, longitude))

Richmond Street, 1, Yonge Street, Downtown Yonge, Toronto Centre, Old Toronto, Toronto, Golden Horseshoe, Ontario, Canada
The geograpical coordinate are 43.6517909, -79.3790667.


In [84]:
richmond_loc = neighborhoods.loc[neighborhoods['Neighborhood'].str.contains('Richmond')].index
richmond_loc

Int64Index([19], dtype='int64')

In [85]:
neighborhoods.loc[richmond_loc,'Latitude'] = latitude
neighborhoods.loc[richmond_loc,'Longitude'] = longitude

In [86]:
neighborhoods.loc[richmond_loc]

,Borough,Neighborhood,Latitude,Longitude
19,Downtown Toronto,Richmond,43.651791,-79.379067


<a id='34'>Map of East, Central, Downtown, and West Toronto Neighborhood</a>  
[Back to Section 3](#ThirdSection)    
Let's recreate the map with the correct Latitude and Longitude:

In [87]:
# create map of Toronto using latitude and longitude values
map_subset = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'],neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_subset)  
    
map_subset

<a id='35'>FourSquare API</a>  
[Back to Section 3](#ThirdSection)    
Now, let's use the foursquare API to explore the neighborhoods and segment them. I am hiding the code since I do not want to show my client ID and client secret to the world.

In [88]:
CLIENT_ID = 'HU0RS0DRV1VJOEOIBR2GEC2P4HPYGHAZZGFESJTN0ADP0GAI' # your Foursquare ID
CLIENT_SECRET = 'GNNJI0DNG1ZVHLTXEDYL541XXNPMZTE040AXNSBDL30X5TEF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

#### Let's explore the first neighborhood in our dataframe.

The name of the neighborhood is:

In [89]:
neighborhoods.loc[0, 'Neighborhood']

'The Danforth West'

Let's get the longitude and latitude of the neighborhood:

In [90]:
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = neighborhoods.loc[0, 'Neighborhood'] # neighborhood name

#### Now, let's get the top 100 venues that are in 'The Danforth West and Riverdale' within a radius of 500 meters.

In [91]:
# define radius
radius = 500
# limit of number of venues returned by Foursquare API
LIMIT = 100
search_query = 'venues'
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    VERSION, 
    radius, 
    LIMIT)
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d35efad6e4650002c4279e6'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'East Danforth',
  'headerFullLocation': 'East Danforth, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 27,
  'suggestedBounds': {'ne': {'lat': 43.6908598045, 'lng': -79.2941044790336},
   'sw': {'lat': 43.681859795499996, 'lng': -79.30652712096641}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b5b9c02f964a520bb0a29e3',
       'name': 'Ted Reeve Arena',
       'location': {'address': '175 Main St.',
        'crossStreet': 'at Gerrard St. E',
        'lat': 43.68452712511952,
        'lng': -79.29942632833531,
        'labeledLatLngs': [{'l

Let's borrow the **get_category_type** function from the Foursquare lab:

In [92]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Let's clean the json and structure it into a *pandas* dataframe.

In [93]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Ted Reeve Arena,Skating Rink,43.684527,-79.299426
1,Beach Hill Smokehouse,BBQ Joint,43.684105,-79.300410
2,Duckworth's Fish and Chips,Fish & Chips Shop,43.688629,-79.299952
3,Cool Runnings,Caribbean Restaurant,43.683497,-79.300017
4,Sultan Shawarma & Falafel,Middle Eastern Restaurant,43.688223,-79.301709


How many venues returned by FourSquare?

In [94]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

27 venues were returned by Foursquare.


## Explore the neighborhoods methodically:

#### Let's borrow the 'getNearbyVenue' function from the lab: 

In [95]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [96]:
# let's put them into one place:
toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

In [97]:
print(toronto_venues.shape)
toronto_venues.head()

(3798, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Danforth West,43.68636,-79.300316,Ted Reeve Arena,43.684527,-79.299426,Skating Rink
1,The Danforth West,43.68636,-79.300316,Beach Hill Smokehouse,43.684105,-79.300410,BBQ Joint
2,The Danforth West,43.68636,-79.300316,Duckworth's Fish and Chips,43.688629,-79.299952,Fish & Chips Shop
3,The Danforth West,43.68636,-79.300316,Cool Runnings,43.683497,-79.300017,Caribbean Restaurant
4,The Danforth West,43.68636,-79.300316,Sultan Shawarma & Falafel,43.688223,-79.301709,Middle Eastern Restaurant


#### Let's find out how many unique categories can be curated from all the returned venues

In [98]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 299 uniques categories.


<a id='36'>Preprocessing venues</a>  
[Back to Section 3](#ThirdSection)    
## Analyze Each Neighborhood

In [99]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

In [100]:
# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
toronto_onehot.head()

,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [101]:
# where does "Neighborhood" located in toronto_onehot dataframe?
toronto_onehot.columns.get_loc('Neighborhood')

194

In [102]:
# saving it to a variable, since we are going to use it multiple times:
num_neigh_to_one= toronto_onehot.columns.get_loc('Neighborhood')

In [103]:
# just for sanity check:
toronto_onehot.columns[num_neigh_to_one]

'Neighborhood'

In [104]:
fixed_columns = [toronto_onehot.columns[num_neigh_to_one]]+list(toronto_onehot.columns[:num_neigh_to_one])+list(toronto_onehot.columns[num_neigh_to_one+1:])

In [105]:
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo
0,The Danforth West,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Danforth West,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Danforth West,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Danforth West,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Danforth West,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Let's see the size:

In [106]:
toronto_onehot.shape

(3798, 299)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [107]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo
0,Adelaide,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,...,0.010000,0.000000,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.000000,0.000000
1,Bathurst Quay,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.04,0.0,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2,Berczy Park,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,...,0.017544,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,Brockton,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,...,0.000000,0.000000,0.000000,0.095238,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
4,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.055556,0.000000
5,CN Tower,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,...,0.000000,0.000000,0.000000,0.000000,0.00,0.011494,0.000000,0.000000,0.011494,0.000000
6,Cabbagetown,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
7,Central Bay Street,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,...,0.011905,0.000000,0.000000,0.000000,0.00,0.011905,0.000000,0.000000,0.011905,0.000000
8,Chinatown,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,...,0.050000,0.010000,0.000000,0.040000,0.00,0.010000,0.000000,0.000000,0.000000,0.000000
9,Christie,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000


#### Let's confirm the new size

In [108]:
toronto_grouped.shape

(72, 299)

#### Let's print each neighborhood along with the top 5 most common venues

In [109]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
         venue  freq
0  Coffee Shop  0.10
1    Gastropub  0.05
2        Hotel  0.05
3   Restaurant  0.05
4         Café  0.04


----Bathurst Quay----
          venue  freq
0   Coffee Shop  0.20
1          Café  0.12
2          Park  0.08
3  Dance Studio  0.04
4         Diner  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.11
1        Cocktail Bar  0.05
2  Seafood Restaurant  0.04
3              Bakery  0.04
4         Cheese Shop  0.04


----Brockton----
                   venue  freq
0                   Park  0.14
1                    Bar  0.14
2  Vietnamese Restaurant  0.10
3            Art Gallery  0.05
4          Grocery Store  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                  venue  freq
0  Gym / Fitness Center  0.06
1                  Park  0.06
2               Brewery  0.06
3                Garden  0.06
4         Garden Center  0.06


----CN Tower----
                venue  freq
0         Coffee Sh

                  venue  freq
0                Garden   1.0
1     Accessories Store   0.0
2          Noodle House   0.0
3  Other Great Outdoors   0.0
4       Organic Grocery   0.0


----Runnymede----
                 venue  freq
0          Pizza Place  0.12
1          Coffee Shop  0.08
2  Rental Car Location  0.04
3             Pharmacy  0.04
4         Burger Joint  0.04


----Ryerson----
                  venue  freq
0           Coffee Shop  0.12
1        Clothing Store  0.05
2      Ramen Restaurant  0.03
3                  Café  0.03
4  Fast Food Restaurant  0.03


----South Hill----
            venue  freq
0  Sandwich Place  0.10
1     Coffee Shop  0.10
2            Café  0.07
3     Pizza Place  0.07
4  History Museum  0.07


----South Niagara----
          venue  freq
0   Coffee Shop  0.05
1   Yoga Studio  0.05
2        Bakery  0.05
3  Optical Shop  0.04
4   Pizza Place  0.04


----St. James Town----
                venue  freq
0         Coffee Shop  0.10
1                Café  0.0

#### Let's put that into a *pandas* dataframe

In [110]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [125]:
num_top_venues = 10
# for: 1st, 2nd, and 3rd
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        # for first, second, third, use this one:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        # others use nth for the column name:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Hotel,Gastropub,Restaurant,Café,Italian Restaurant,Cosmetics Shop,Breakfast Spot,Japanese Restaurant,American Restaurant
1,Bathurst Quay,Coffee Shop,Café,Park,Grocery Store,Dance Studio,Garden,Sculpture Garden,Diner,Caribbean Restaurant,Japanese Restaurant
2,Berczy Park,Coffee Shop,Cocktail Bar,Steakhouse,Seafood Restaurant,Beer Bar,Café,Farmers Market,Cheese Shop,Bakery,Restaurant
3,Brockton,Bar,Park,Vietnamese Restaurant,Pizza Place,French Restaurant,Gastropub,Korean Restaurant,Dive Bar,South American Restaurant,Jazz Club
4,Business Reply Mail Processing Centre 969 Eastern,Comic Shop,Brewery,Auto Workshop,Light Rail Station,Burrito Place,Restaurant,Recording Studio,Garden,Park,Gym / Fitness Center


<a id='37'>K-Means Clustering</a>  
[Back to Section 3](#ThirdSection)    
## Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 10 clusters.

In [126]:
# set number of clusters
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([7, 7, 7, 0, 0, 7, 0, 7, 0, 7], dtype=int32)

In [127]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,The Danforth West,43.686360,-79.300316,0,Pharmacy,Coffee Shop,Grocery Store,Bus Line,BBQ Joint,Sandwich Place,Café,Bakery,Sushi Restaurant,Baseball Field
1,East Toronto,Riverdale,43.665470,-79.352594,0,Chinese Restaurant,Vietnamese Restaurant,Bakery,Light Rail Station,Grocery Store,Burger Joint,Bar,Asian Restaurant,Baseball Field,Gym / Fitness Center
2,East Toronto,India Bazaar,43.672223,-79.323503,0,Indian Restaurant,Café,Grocery Store,Donut Shop,Sandwich Place,Bus Stop,Diner,Burger Joint,Theater,Asian Restaurant
3,East Toronto,The Beaches West,43.671024,-79.296712,0,Park,Beach,Coffee Shop,Pizza Place,Breakfast Spot,Pub,Bar,Tea Room,Thai Restaurant,Japanese Restaurant
4,Central Toronto,Summerhill East,43.681678,-79.390504,0,Italian Restaurant,Sushi Restaurant,Pizza Place,Coffee Shop,Café,Spa,Beer Store,Sporting Goods Shop,Bakery,Bank


<a id='38'>Cluster Map</a>  
[Back to Section 3](#ThirdSection)    
Finally, let's visualize the resulting clusters

In [128]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='39'>Examine Cluster</a>  
[Back to Section 3](#ThirdSection)    

Now, let's examine each cluster and determine the discriminating venue categories that distinguish each cluster. 

#### Cluster 1

In [132]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[0,1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,The Danforth West,Pharmacy,Coffee Shop,Grocery Store,Bus Line,BBQ Joint,Sandwich Place,Café,Bakery,Sushi Restaurant,Baseball Field
1,East Toronto,Riverdale,Chinese Restaurant,Vietnamese Restaurant,Bakery,Light Rail Station,Grocery Store,Burger Joint,Bar,Asian Restaurant,Baseball Field,Gym / Fitness Center
2,East Toronto,India Bazaar,Indian Restaurant,Café,Grocery Store,Donut Shop,Sandwich Place,Bus Stop,Diner,Burger Joint,Theater,Asian Restaurant
3,East Toronto,The Beaches West,Park,Beach,Coffee Shop,Pizza Place,Breakfast Spot,Pub,Bar,Tea Room,Thai Restaurant,Japanese Restaurant
4,Central Toronto,Summerhill East,Italian Restaurant,Sushi Restaurant,Pizza Place,Coffee Shop,Café,Spa,Beer Store,Sporting Goods Shop,Bakery,Bank
6,Central Toronto,Rathnelly,Park,Italian Restaurant,Mexican Restaurant,French Restaurant,American Restaurant,Liquor Store,Electronics Store,BBQ Joint,Pizza Place,Coffee Shop
8,Central Toronto,South Hill,Coffee Shop,Sandwich Place,History Museum,Café,Pizza Place,Park,Pub,Castle,Jewish Restaurant,Burger Joint
9,Central Toronto,Summerhill West,Italian Restaurant,Sushi Restaurant,Pizza Place,Coffee Shop,Café,Spa,Beer Store,Sporting Goods Shop,Bakery,Bank
11,Downtown Toronto,Cabbagetown,Restaurant,Coffee Shop,Café,Indian Restaurant,Diner,Beer Store,Japanese Restaurant,Italian Restaurant,Gastropub,Bakery
29,Central Toronto,The Annex,Pizza Place,Coffee Shop,Ice Cream Shop,Thai Restaurant,Park,Gym,Bistro,Grocery Store,Indian Restaurant,Donut Shop


It seems that `cluster 1` has a lot of places to hang out and eat.

#### Cluster 2

In [133]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[0,1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
69,Central Toronto,Roselawn,Garden,Zoo,Event Space,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant


Cluster 2 has only one neighborhood with gathering places such as garden, zoo, and event space as the first three venues.

#### Cluster 3

In [134]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[0,1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Central Toronto,Forest Hill SE,Accessories Store,Playground,Bank,Event Space,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant
27,Central Toronto,Forest Hill West,Accessories Store,Playground,Bank,Event Space,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant
28,Central Toronto,Forest Hill North,Accessories Store,Playground,Bank,Event Space,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant


Cluster 3 is located in the Central Toronto Borough and is in the Forest Hill area.

#### Cluster 4

In [135]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[0,1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Central Toronto,Moore Park,Playground,Gym,Zoo,Ethiopian Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store


Cluster 4 has one neighborhood (Moore Park) with playground, gym, and zoo as the three most common venues.

#### Cluster 5

In [136]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[0,1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
56,West Toronto,Swansea,Dance Studio,Skating Rink,Pilates Studio,Park,Bus Line,Electronics Store,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant


Cluster 5 has one neighborhood (Swansea) with dance studio, skating rink, and pilates studio as the three most common venues.

#### Cluster 6

In [137]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[0,1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
64,Downtown Toronto,Rosedale,Park,Playground,Building,Trail,Zoo,Egyptian Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant


Cluster 6 has one neighborhood (Rosedal) with park, playground, and building as the three most common venues.

#### Cluster 7

In [138]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[0,1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
60,Central Toronto,Lawrence Park,Bus Line,Park,Swim School,Ethiopian Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store


Lawrence Park is the one neighborhood for Cluster 7 with bus line, park, and swim school as the three most common venues.

#### Cluster 8

In [139]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 7, toronto_merged.columns[[0,1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Central Toronto,Deer Park,Coffee Shop,Italian Restaurant,Pub,Sushi Restaurant,Café,Grocery Store,Bagel Shop,Restaurant,Pizza Place,Sandwich Place
12,Downtown Toronto,St. James Town,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Pizza Place,Bakery,Breakfast Spot,Beer Bar,Cosmetics Shop
13,Downtown Toronto,Harbourfront,Coffee Shop,Restaurant,Hotel,Café,Italian Restaurant,Pizza Place,Gym,Music Venue,Sporting Goods Shop,Sushi Restaurant
14,Downtown Toronto,Regent Park,Coffee Shop,Thai Restaurant,Beer Store,Animal Shelter,Sushi Restaurant,Fast Food Restaurant,Auto Dealership,Food Truck,Restaurant,Moving Target
15,Downtown Toronto,Ryerson,Coffee Shop,Clothing Store,Fast Food Restaurant,Café,Middle Eastern Restaurant,Ramen Restaurant,Japanese Restaurant,Sandwich Place,Tea Room,Burger Joint
16,Downtown Toronto,Garden District,Coffee Shop,Clothing Store,Fast Food Restaurant,Cosmetics Shop,Restaurant,Hotel,Middle Eastern Restaurant,Theater,Tea Room,Plaza
17,Downtown Toronto,King,Coffee Shop,Restaurant,Hotel,Italian Restaurant,Café,Gastropub,American Restaurant,Japanese Restaurant,Gym,Deli / Bodega
18,Downtown Toronto,Adelaide,Coffee Shop,Hotel,Gastropub,Restaurant,Café,Italian Restaurant,Cosmetics Shop,Breakfast Spot,Japanese Restaurant,American Restaurant
19,Downtown Toronto,Richmond,Coffee Shop,Cosmetics Shop,Gastropub,Hotel,Italian Restaurant,Restaurant,Clothing Store,Café,Plaza,American Restaurant
20,Downtown Toronto,Harbourfront East,Coffee Shop,Restaurant,Hotel,Café,Italian Restaurant,Pizza Place,Gym,Music Venue,Sporting Goods Shop,Sushi Restaurant


Cluster 8 has either Coffee Shop or Cafe as the first two most common venue. 

#### Cluster 9

In [140]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 8, toronto_merged.columns[[0,1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Downtown Toronto,Toronto Islands,American Restaurant,Playground,Beer Garden,Scenic Lookout,Beach,Farm,Event Space,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant


Cluster 9 is Toronto Islands which is off the coast of Downtown Toronto and has american restaurant, playground, and beer garden as the three most common venue.

#### Cluster 10

In [141]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 9, toronto_merged.columns[[0,1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
58,East Toronto,The Beaches,Other Great Outdoors,Pub,Health Food Store,Trail,Egyptian Restaurant,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant


Cluster 10 is The Beaches located in East Toronto Borough with great outdoors, pub, and health food store as the three most common venues. 